In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import train 
import data_utils

DuplicateFlagError: The flag 'dataset_params' is defined twice. First from NJODE.data_utils, Second from data_utils.  Description from first occurrence: name of the dict with data hyper-params

In [3]:
dataset_dic = {
    'drift': np.array([0.5, 0.5, 0.5]).tolist(),
    'volatility': np.array([[0.2, 0.1, 0.1], [0.1, 0.2, 0.1], [0.1,0.1, 0.2]]).tolist(),
    'mean': np.array([0.5, 0.5, 0.5]).tolist(),
    'poisson_lambda': 3.0,
    'speed': np.array([[0.3, 0.0, 0.0], [0.0, 0.4, 0.0],[0.0,0.0,0.5]]).tolist(),
    'correlation': 0.5,
    'nb_paths': 300,
    'nb_steps': 100,
    'S0': np.array([1.0, 1.5 ,2.0]).tolist(),
    'maturity': 1.0,
    'dimension': 3,
    'obs_perc': 1.0,
    'scheme': 'euler',
    'return_vol': False,
    'v0': 1,
    'hurst': 0.75,
    'FBMmethod': 'daviesharte',
    'model_name': 'OrnsteinUhlenbeckWithCrossTerms',
    'dt': 0.01
}

In [4]:
datasetpath, dataset_id = data_util.create_dataset("OrnsteinUhlenbeckWithCrossTerms",dataset_dic)

In [5]:
dataset_id

30

In [9]:
dataset_metadata = data_util.load_metadata(stock_model_name="OrnsteinUhlenbeckWithCrossTerms",time_id=30)

In [10]:
dataset_metadata["dimension"]

3

In [ ]:
train.train(dataset="OrnsteinUhlenbeckWithCrossTerms", dataset_id=30, data_dict=dataset_dic, use_cond_exp = False)

In [7]:
from models import NJODE
checkpoint = torch.load("../data/saved_models/id-20/best_checkpoint/checkpt.tar")
state_dict=checkpoint['model_state_dict']

In [8]:
state_dict

OrderedDict([('ode_f.f.0.weight',
              tensor([[-4.6704e-02,  7.2204e-02, -6.2405e-02,  ..., -7.8976e-03,
                        3.3973e-02,  7.3963e-03],
                      [-1.4619e-02, -4.0299e-02,  2.2504e-02,  ..., -3.4302e-02,
                       -3.6700e-03, -5.6320e-02],
                      [ 1.9629e-02,  2.2321e-02, -2.4921e-02,  ...,  1.8087e-02,
                        3.9124e-03, -1.8601e-05],
                      ...,
                      [ 6.8489e-02,  6.8699e-02,  3.7755e-02,  ...,  3.0108e-03,
                        7.9594e-03, -5.2973e-02],
                      [-1.7247e-02,  4.2316e-02, -1.2825e-02,  ..., -2.7936e-02,
                       -7.3024e-03, -1.5222e-02],
                      [-1.5287e-02, -1.9622e-02, -7.2343e-02,  ..., -5.9798e-03,
                       -4.6128e-03, -4.7666e-09]])),
             ('ode_f.f.0.bias',
              tensor([-0.0348, -0.0119,  0.0237, -0.0226,  0.0337,  0.0263, -0.0170,  0.0350,
                       0

In [9]:
options = {'options': {'which_loss': 'standard'}}

In [10]:
ode_nn=((50, "tanh"), (50, "tanh"))
readout_nn=((50, "tanh"), (50, "tanh"))
enc_nn=((50, "tanh"), (50, "tanh"))

In [11]:
params_dict = {  # create a dictionary of the wanted parameters
    'input_size': 9, 'epochs': 100,'hidden_size': 10, 'output_size': 9,
    'ode_nn': ode_nn, 'readout_nn': readout_nn, 'enc_nn': enc_nn,
    'use_rnn': False,
    'options': options, "input_coords":np.arange(9), "output_coords":np.arange(9)
}

In [12]:
model=NJODE(**params_dict)

using loss: standard
neuralODE use input scaling with tanh
use residual network: input_size=9, output_size=10
use residual network: input_size=10, output_size=9


In [13]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [20]:
times=[]
time_ptr = [0]
obs_idx = []
delta_t=0.01
T=0.01
X= torch.empty(9,9)
start_X= torch.tensor([[1.0], [1.5], [2.0], [1.0], [1.5**2], [4.0], [1.5], [2.0],[3.0]]).reshape(1,9)
n_obs_ot=torch.empty((1,1))

In [21]:
prediction=model.get_pred(times, time_ptr, X, obs_idx, delta_t, T, start_X, n_obs_ot)
prediction["pred"].size()

torch.Size([101, 1, 9])

In [26]:
start_X[:, :3]

tensor([[1.0000, 1.5000, 2.0000]])

In [30]:
prediction["pred"][1, :, :3]

tensor([[1.0021, 1.5033, 1.9849]], grad_fn=<SliceBackward0>)

In [35]:
def get_mu_hat(times,time_ptr,obs_idx,delta_t,T,X,start_X, n_obs_ot, d):
    njode_pred=model.get_pred(times, time_ptr, X, obs_idx, delta_t, T, start_X, n_obs_ot)["pred"]
    mu=(njode_pred[-1,:,:d]-start_X[:,:d])/delta_t
    return mu

In [39]:
def construct_matrix(output, d):
    XX = torch.zeros((d, d), dtype=output.dtype)

    for i in range(d):
        XX[i, i] = output[:,d + i]

    index = 2 * d
    for i in range(d):
        for j in range(i + 1, d):
            XX[i, j] = output[:,index]
            XX[j, i] = output[:,index]
            index += 1

    return XX

In [112]:
def get_sigma_hat(times,time_ptr,obs_idx,delta_t,T,X,start_X, n_obs_ot,d):
    X0=start_X[:, :d]
    njode_pred=model.get_pred(times, time_ptr, X, obs_idx, delta_t, T, start_X, n_obs_ot)["pred"]
    sig1=(construct_matrix(njode_pred[-1,:],d)-construct_matrix(start_X,d))/delta_t
    mu = get_mu_hat(times,time_ptr,obs_idx,delta_t,T,X,start_X, n_obs_ot,d)
    sig2=X0.T@mu+mu.T@X0
    return sig1-sig2 

In [36]:
mu=get_mu_hat(times,time_ptr,obs_idx,delta_t,T,X,start_X, n_obs_ot, 3)

In [37]:
mu

tensor([[ 0.2109,  0.3317, -1.5054]], grad_fn=<DivBackward0>)

In [108]:
sigma=get_sigma_hat(times,time_ptr,obs_idx,delta_t,T,X,start_X, n_obs_ot,3)

In [109]:
sigma

tensor([[-0.4218, -0.6480,  1.0836],
        [-0.6480, -0.9950,  1.5947],
        [ 1.0836,  1.5947,  6.0214]], grad_fn=<NegBackward0>)

In [110]:
x=sigma.detach().numpy()

In [111]:
np.linalg.eigvals(x)

array([ 6.4914680e+00,  3.3932055e-16, -1.8868544e+00], dtype=float32)

In [74]:
 np.linalg.cholesky(x)

LinAlgError: Matrix is not positive definite

In [77]:
input=construct_matrix(start_X,3)

In [78]:
input

tensor([[1.0000, 1.5000, 2.0000],
        [1.5000, 2.2500, 3.0000],
        [2.0000, 3.0000, 4.0000]])

In [65]:
input=input.detach().numpy()

In [66]:
np.linalg.eigvals(input)

array([0.  , 7.25, 0.  ], dtype=float32)

In [91]:
njode_pred=model.get_pred(times, time_ptr, X, obs_idx, delta_t, 0.5, start_X, n_obs_ot)["pred"]
output=construct_matrix(njode_pred[-1,:],3)